In [7]:
import numpy as np
import pandas as pd
import sklearn as skl
import random
import matplotlib.pyplot as plt
import re

%matplotlib inline

# Global Variables

In [4]:
seed = 2293687334
random.seed(seed)
np.random.seed(seed)

# Import

In [5]:
#  load product tf-idf vectors
prods_original = pd.read_excel('/home/tuna/Projects/CS673/amazon_api/dumps/vectorized_products.xlsx')
prods_original.head()

,name,features,price,avg_score,review_vol,description,feature_vector
1,CAT5 TO BNC Passive Video and Power Balun Tran...,"5 Cameras, CAT5 TO BNC Passive Video and Power...",$13.00,5.000000,1,cat5 to bnc passive video and power balun tran...,[ 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...
4,Fujifilm S2100HD 10.0 Megapixel 15x Optical/5....,Allow 15x zoom ups during your recordings.\nA ...,$210.93,5.000000,6,fujifilm s2100hd 10.0 megapixel 15x optical 5....,[ 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...
5,Nikon D3100 Digital SLR Camera & 18-55mm VR Le...,KIT INCLUDES 8 PRODUCTS -- All Brand New Items...,$469.95,4.760563,71,nikon d3100 digital slr camera 18-55mm vr le...,[ 3. 1. 0. 0. 0. 2. 0. 0. 0. 4. 0. ...
6,Olympus FE-46 12MP Digital Camera Blue Crayola...,5x optical zoom; 12-megapixel resolution for p...,$149.99,3.846154,13,olympus fe-46 12mp digital camera blue crayola...,[ 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. ...
8,Kodak Slice 14MP Digital Camera with Schneider...,14-megapixel resolution for high-quality pictu...,$299.00,3.694915,59,kodak slice 14mp digital camera with schneider...,[ 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...


In [15]:
#  import new product proposals
new_prods_original = pd.read_excel("/home/tuna/Projects/CS673/Product_Ideation_Bot/new_product_proposals.xlsx")
new_prods_original.head()

,0,1,2,3,4,5,6,7,8,9,...,110,111,112,113,114,115,116,117,118,119
0,1,0,0,0,0,1,0,0,0,1,...,0,1,0,0,0,0,0,1,0,1
1,0,0,1,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,1,0,1
2,0,0,0,0,0,1,0,0,0,1,...,0,1,0,0,0,0,0,1,0,1
3,0,0,1,0,0,0,0,0,0,0,...,0,1,1,0,1,1,1,1,0,1
4,1,0,1,0,0,1,0,0,0,1,...,0,1,0,0,0,0,0,1,0,1


In [393]:
list(new_prods_original.sum(axis=0))

[31,
 1,
 89,
 1,
 2,
 73,
 1,
 10,
 3,
 92,
 3,
 1,
 4,
 1,
 4,
 7,
 1,
 3,
 4,
 22,
 1,
 4,
 1,
 3,
 7,
 6,
 2,
 1,
 3,
 3,
 4,
 7,
 1,
 2,
 5,
 12,
 4,
 2,
 4,
 1,
 19,
 7,
 4,
 2,
 4,
 7,
 2,
 1,
 4,
 2,
 11,
 7,
 4,
 4,
 4,
 1,
 4,
 6,
 15,
 1,
 4,
 2,
 4,
 10,
 2,
 8,
 1,
 7,
 118,
 13,
 1,
 8,
 1,
 2,
 4,
 1,
 2,
 4,
 9,
 1,
 0,
 5,
 1,
 14,
 4,
 40,
 0,
 1,
 35,
 2,
 6,
 4,
 3,
 2,
 11,
 8,
 2,
 6,
 90,
 1,
 55,
 136,
 11,
 8,
 5,
 24,
 56,
 3,
 120,
 53,
 6,
 168,
 103,
 1,
 44,
 68,
 45,
 179,
 2,
 179]

# Clean

In [14]:
#  extract vector for x, some aggregate of avg_score, price, and review vol for y
prod_vectors_x = prods_original['feature_vector'].as_matrix()
prod_vectors_x = list(prod_vectors_x.astype('str'))
tmp = []
for prod_str in prod_vectors_x:
    tmp_array = np.asarray(re.split("\s+", prod_str.replace('\n', ' ')[1:-1].strip())).astype(float)
    tmp.append(tmp_array)
prod_vectors_x = tmp
prod_vectors_x = np.atleast_2d(prod_vectors_x)

#  change to binary values
prod_vectors_x = (prod_vectors_x > 0).astype(int)
prod_vectors_x

array([[0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 1, 0, 1],
       [1, 1, 0, ..., 1, 0, 1],
       ..., 
       [0, 0, 0, ..., 1, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1]])

In [404]:
#  create prod_vectors_y (price, avg score, reviw vol)
price_list = prods_original['price']
score_list = prods_original['avg_score']
vol_list = prods_original['review_vol']


# #  TODO: need to define what important to us, this values total revenue above all us
# def calculate_value(price, score, vol):
#     price = price.replace(',','')
#     price = float(price.split('.')[0][1:])
#     score = float(score)
#     vol = float(vol)
#     total_revenue = price * vol
#     score_multiplier = score / 2.5 #  this way the score can either doble or fractionalize the value
#     return total_revenue * score_multiplier
    

#  TODO: need to define what important to us, this values total revenue above all us
def calculate_value(price, score, vol):
#     price = price.replace(',','')
#     price = float(price.split('.')[0][1:])
    score = float(score)
    vol = float(vol)
    return 1.0 + (score - 1.0)*(1.0 - 1.0/(vol/10.0 + 1.0))

In [406]:
prod_vectors_y = [calculate_value(price_list.iloc[i], score_list.iloc[i], vol_list.iloc[i]) \
                  for i in range(len(price_list))]
prod_vectors_y

[1.3636363636363638,
 2.5,
 4.296296296296296,
 2.6086956521739126,
 3.3043478260869565,
 1.7857142857142856,
 1.3636363636363638,
 3.8771929824561404,
 1.6153846153846156,
 3.08,
 3.2419354838709675,
 2.25,
 2.2666666666666666,
 1.9230769230769234,
 2.3125,
 1.3636363636363638,
 2.1764705882352944,
 1.3636363636363638,
 3.328125,
 4.520884520884521,
 1.3636363636363638,
 3.6799276672694394,
 1.3636363636363638,
 3.076923076923077,
 3.2325581395348837,
 1.3333333333333333,
 2.9948849104859336,
 3.6069651741293534,
 4.482587064676617,
 3.603960396039604,
 3.341463414634146,
 3.7222222222222223,
 1.3636363636363638,
 1.3636363636363638,
 1.3636363636363638,
 4.226190476190476,
 1.272727272727273,
 3.038961038961039,
 3.208333333333333,
 2.8076923076923075,
 4.5310136157337375,
 1.3125,
 4.011494252873563,
 2.708333333333333,
 4.133620689655173,
 1.3636363636363638,
 2.8055555555555554,
 2.333333333333333,
 2.1764705882352944,
 1.6666666666666665,
 1.3636363636363638,
 1.3636363636363638,

In [407]:
#  train/test split on prod_vectors
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(prod_vectors_x, prod_vectors_y, test_size=0.2)

In [408]:
#  convert new_prods to numpy matrix
new_prod_vectors_x = new_prods_original.as_matrix()
new_prod_vectors_x

array([[1, 0, 0, ..., 1, 0, 1],
       [0, 0, 1, ..., 1, 0, 1],
       [0, 0, 0, ..., 1, 0, 1],
       ..., 
       [1, 0, 1, ..., 1, 0, 0],
       [0, 0, 1, ..., 1, 0, 1],
       [0, 0, 0, ..., 1, 0, 1]])

#  Model

In [409]:
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.ensemble import GradientBoostingRegressor as GBR
from sklearn.neural_network import MLPRegressor as MLPR
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression as LR
from sklearn.linear_model import Lasso 
from sklearn.linear_model import Perceptron 

## Experimenting with several models

In [410]:
model = RFR(n_jobs=-1, oob_score=True, n_estimators=100, max_depth=None, max_features=2)
model.fit(X_train, y_train)
model.score(X_train, y_train)

0.89605544704059836

In [411]:
model.score(X_test, y_test)

0.3929408246633736

In [412]:
model = GBR(n_estimators=10000, max_depth=4, max_features=15, learning_rate=0.2)
# model = GBR(n_estimators=100, max_depth=3, max_features='auto')
model.fit(X_train, y_train)
model.score(X_train, y_train)

0.9379616911679185

In [413]:
model.score(X_test, y_test)

0.18897269320035862

In [414]:
model = SVR(kernel='poly', C=0.0001, epsilon=0.0001)
# model = GBR(n_estimators=100, max_depth=3, max_features='auto')
model.fit(X_train, y_train)
model.score(X_train, y_train)

-0.00012313638858141296

In [415]:
model.score(X_test, y_test)

7.9628761014971516e-06

In [416]:
model = LR()
# model = GBR(n_estimators=100, max_depth=3, max_features='auto')
model.fit(X_train, y_train)
model.score(X_train, y_train)

0.29518804063149962

In [417]:
model.score(X_test, y_test)

0.25200188477963759

In [418]:
model = Lasso()
# model = GBR(n_estimators=100, max_depth=3, max_features='auto')
model.fit(X_train, y_train)
model.score(X_train, y_train)

0.0

In [419]:
model.score(X_test, y_test)

-0.00011218310045579825

###  Final Model

In [420]:
model = RFR(n_jobs=-1, oob_score=True, n_estimators=100, max_depth=None, max_features=2)
model.fit(prod_vectors_x, prod_vectors_y)
model.score(prod_vectors_x, prod_vectors_y)

0.89439639209620503

#  Predict New Product Scores and Rank

In [421]:
new_prod_scores = model.predict(new_prod_vectors_x)
new_prod_scores

array([ 2.12328397,  2.20211114,  2.15781002,  3.21685338,  2.08659042,
        2.09400616,  2.57579619,  2.54178748,  2.11276423,  1.96766683,
        1.78116459,  2.91377243,  1.78116459,  3.38993423,  3.19393102,
        3.0664508 ,  2.96398389,  2.15781002,  2.15781002,  2.54178748,
        2.02404828,  2.10968246,  3.23353242,  1.96766683,  2.14044319,
        2.11276423,  2.18162834,  2.00187815,  2.02404828,  2.15781002,
        2.15781002,  2.15781002,  2.14410551,  2.15781002,  2.53312491,
        2.15781002,  2.59734773,  3.59508463,  2.10213497,  3.19393102,
        2.211713  ,  2.12310134,  2.27011407,  2.54178748,  2.94746455,
        2.96398389,  2.07260449,  2.47200728,  2.07260449,  3.59508463,
        2.716213  ,  2.45698186,  2.18463338,  2.14044319,  2.96398389,
        2.54178748,  2.15781002,  2.51013106,  2.12310134,  2.54178748,
        1.48420945,  2.02404828,  2.09110936,  3.06279437,  3.33445726,
        2.96398389,  2.50593283,  2.20211114,  2.18463338,  2.11

In [422]:
new_prod_combined_scores = np.hstack((new_prod_vectors_x, np.atleast_2d(new_prod_scores).T))
top_ranked_prods = np.asarray(sorted(new_prod_combined_scores, key=lambda x: -x[-1]))

In [423]:
new_prod_combined_scores[:,-1]

array([ 2.12328397,  2.20211114,  2.15781002,  3.21685338,  2.08659042,
        2.09400616,  2.57579619,  2.54178748,  2.11276423,  1.96766683,
        1.78116459,  2.91377243,  1.78116459,  3.38993423,  3.19393102,
        3.0664508 ,  2.96398389,  2.15781002,  2.15781002,  2.54178748,
        2.02404828,  2.10968246,  3.23353242,  1.96766683,  2.14044319,
        2.11276423,  2.18162834,  2.00187815,  2.02404828,  2.15781002,
        2.15781002,  2.15781002,  2.14410551,  2.15781002,  2.53312491,
        2.15781002,  2.59734773,  3.59508463,  2.10213497,  3.19393102,
        2.211713  ,  2.12310134,  2.27011407,  2.54178748,  2.94746455,
        2.96398389,  2.07260449,  2.47200728,  2.07260449,  3.59508463,
        2.716213  ,  2.45698186,  2.18463338,  2.14044319,  2.96398389,
        2.54178748,  2.15781002,  2.51013106,  2.12310134,  2.54178748,
        1.48420945,  2.02404828,  2.09110936,  3.06279437,  3.33445726,
        2.96398389,  2.50593283,  2.20211114,  2.18463338,  2.11

In [424]:
top_ranked_words

array(['design', 'storage', 'canon', 'support', 'viewing', 'case',
       'smartphone', '11', '32gb', 'slr', 'light', 'network', 'speed',
       'ram', 'pc', 'cell', 'windows', 'technology', 'vision', 'movie',
       'touchscreen', 'hdtv', 'free', 'detection', 'monitor', 'control',
       'features', '18', '1800', 'view', 'easy', '1900', 'ghz', 'ccd',
       'mode', '14', 'dvr', 'hard', 'compatible', 'quad', 'sdhc', '900',
       'sony', 'cameras', 'power', 'cable', 'drive', 'samsung', 'night',
       'audio', '720p', '850', 'capture', 'motion', 'surveillance',
       'white', 'bluetooth', '3g', 'auto', 'warranty', 'flash', 'new',
       'images', 'remote', 'hdmi', 'smart', 'usb', 'tv', 'kit', '12',
       'time', 'gsm', 'core', 'color', 'security', 'wireless', 'gb',
       'dual', 'recording', 'touch', 'wi', 'includes', 'angle', 'wide',
       'sd', 'cmos', 'fi', 'mobile', '10', 'quality', 'included', '1080p',
       'android', 'built', 'processor', 'unlocked', 'led', '16', 'battery',

#  Inspect Top Ranked Products

In [431]:
new_prod_combined_scores.shape

(200, 121)

In [425]:
top_ranked_words_df = pd.read_csv('/home/tuna/Projects/CS673/amazon_api/dumps/top_ranked_words.csv')
top_ranked_words = top_ranked_words_df['0'].as_matrix()


def get_words_from_vec(vec, top_ranked_words):
    bool_vec = vec > 0
    print(set(top_ranked_words[bool_vec]))
# get_words_from_vec_vectorized = np.vectorize(get_words_from_vec)

In [426]:
top_ranked_words[new_prod_combined_scores[150,:-1] > 0]

/home/tuna/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel/__main__.py:1: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 1000 but corresponding boolean dimension is 120
  if __name__ == '__main__':


array(['varifocal', '3ghz', 'tech', 'end', 'ericsson', 'browsing',
       'formats', 'contact', 'disk', 'curve', 'preview'], dtype=object)

In [437]:
top_ranked_words[new_prod_combined_scores[0,:-1] > 0]

/home/tuna/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel/__main__.py:1: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 1000 but corresponding boolean dimension is 120
  if __name__ == '__main__':


array(['brown', '3ghz', 'tech', 'end', 'grade', 'ericsson', 'browsing',
       'formats', 'contact', 'curve', 'preview'], dtype=object)

In [439]:
top_ranked_words[new_prod_combined_scores[5,:-1] > 0]

/home/tuna/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel/__main__.py:1: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 1000 but corresponding boolean dimension is 120
  if __name__ == '__main__':


array(['brown', 'varifocal', '3ghz', 'tech', '65ft', '2010', 'room', 'end',
       '640x480', '600hz', 'grade', 'ericsson', 'browsing', 'formats',
       'contact', 'curve', 'preview'], dtype=object)

In [440]:
top_ranked_words[new_prod_combined_scores[92,:-1] > 0]

/home/tuna/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel/__main__.py:1: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 1000 but corresponding boolean dimension is 120
  if __name__ == '__main__':


array(['optics', 'approximately', 'disk', 'right', 'longer', 'clips'], dtype=object)

In [442]:
top_ranked_words[new_prod_combined_scores[132,:-1] > 0]

/home/tuna/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel/__main__.py:1: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 1000 but corresponding boolean dimension is 120
  if __name__ == '__main__':


array(['varifocal', '3ghz', 'tech', 'end', 'let', 'browsing', 'formats',
       '4ch', 'contact', 'disk', 'curve', 'preview'], dtype=object)

In [443]:
top_ranked_words[new_prod_combined_scores[178,:-1] > 0]

/home/tuna/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel/__main__.py:1: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 1000 but corresponding boolean dimension is 120
  if __name__ == '__main__':


array(['2600', 'sticker', 'greater', 'smartphones', 'incredible',
       'platform', 'accelerometer', 'date'], dtype=object)

In [444]:
top_ranked_words[new_prod_combined_scores[199,:-1] > 0]

/home/tuna/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel/__main__.py:1: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 1000 but corresponding boolean dimension is 120
  if __name__ == '__main__':


array(['3ghz', 'tech', 'end', 'ericsson', 'browsing', 'formats', 'contact',
       'curve', 'preview'], dtype=object)

In [445]:
top_ranked_words[new_prod_combined_scores[104,:-1] > 0]

/home/tuna/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel/__main__.py:1: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 1000 but corresponding boolean dimension is 120
  if __name__ == '__main__':


array(['a6', 'gmail', 'fix', 'el14', 'male', 'faces', 'poe', 'optics',
       'ex', 'approximately', '4ch', 'contact', 'disk', 'right', 'longer',
       'clips'], dtype=object)

In [446]:
top_ranked_words[new_prod_combined_scores[98,:-1] > 0]

/home/tuna/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel/__main__.py:1: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 1000 but corresponding boolean dimension is 120
  if __name__ == '__main__':


array(['optics', 'approximately', 'disk', 'right', 'longer', 'clips',
       'curve', 'preview'], dtype=object)

In [447]:
top_ranked_words[new_prod_combined_scores[32,:-1] > 0]

/home/tuna/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel/__main__.py:1: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 1000 but corresponding boolean dimension is 120
  if __name__ == '__main__':


array(['end', 'browsing', 'formats', 'contact', 'disk', 'curve', 'preview'], dtype=object)

In [441]:
top_ranked_words[new_prod_combined_scores[111,:-1] > 0]

/home/tuna/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel/__main__.py:1: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 1000 but corresponding boolean dimension is 120
  if __name__ == '__main__':


array(['browsing', 'date', 'preview'], dtype=object)

In [427]:
top_ranked_words[new_prod_combined_scores[140,:-1] > 0]

/home/tuna/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel/__main__.py:1: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 1000 but corresponding boolean dimension is 120
  if __name__ == '__main__':


array(['end', 'ericsson', 'browsing', 'formats', 'contact', 'curve',
       'preview'], dtype=object)

In [428]:
top_ranked_words[new_prod_combined_scores[23,:-1] > 0]

/home/tuna/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel/__main__.py:1: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 1000 but corresponding boolean dimension is 120
  if __name__ == '__main__':


array(['brown', 'varifocal', '3ghz', 'srs', 'tech', '65ft', '2010', 'room',
       'end', '640x480', '600hz', 'grade', 'ericsson', 'browsing',
       'formats', 'contact', 'curve', 'preview'], dtype=object)

In [432]:
get_words_from_vec(new_prod_combined_scores[168,:-1], top_ranked_words)

set(['preview', 'longer', 'curve', 'approximately', 'contact', 'disk', 'optics'])


/home/tuna/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel/__main__.py:7: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 1000 but corresponding boolean dimension is 120


In [433]:
get_words_from_vec(new_prod_combined_scores[123,:-1], top_ranked_words)

set(['preview', 'end', 'varifocal', 'browsing', 'curve', 'contact', 'let', '4ch', 'formats', 'tech', 'disk'])


/home/tuna/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel/__main__.py:7: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 1000 but corresponding boolean dimension is 120


In [434]:
get_words_from_vec(new_prod_combined_scores[12,:-1], top_ranked_words)

set(['end', 'browsing', 'curve', 'formats', 'preview', 'ericsson'])


/home/tuna/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel/__main__.py:7: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 1000 but corresponding boolean dimension is 120


In [435]:
get_words_from_vec(new_prod_combined_scores[49,:-1], top_ranked_words)

set(['clips', 'poe', 'longer', 'varifocal', 'a6', 'curve', 'approximately', 'disk', 'contact', 'right', 'ex', '4ch', 'faces', 'el14', 'preview', 'male', 'optics', 'fix'])


/home/tuna/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel/__main__.py:7: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 1000 but corresponding boolean dimension is 120


In [436]:
get_words_from_vec(new_prod_combined_scores[1,:-1], top_ranked_words)

set(['preview', 'end', 'varifocal', 'browsing', 'curve', 'contact', 'formats', 'disk'])


/home/tuna/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel/__main__.py:7: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 1000 but corresponding boolean dimension is 120


In [382]:
get_words_from_vec(new_prod_combined_scores[89,:-1], top_ranked_words)

set(['case', 'battery', 'digital', 'canon', 'zoom', 'slr', 'kit', 'lens', 'camera', 'card', 'memory', 'cmos', 'mp'])


In [381]:
top_ranked_words

array(['design', 'storage', 'canon', 'support', 'viewing', 'case',
       'smartphone', '11', '32gb', 'slr', 'light', 'network', 'speed',
       'ram', 'pc', 'cell', 'windows', 'technology', 'vision', 'movie',
       'touchscreen', 'hdtv', 'free', 'detection', 'monitor', 'control',
       'features', '18', '1800', 'view', 'easy', '1900', 'ghz', 'ccd',
       'mode', '14', 'dvr', 'hard', 'compatible', 'quad', 'sdhc', '900',
       'sony', 'cameras', 'power', 'cable', 'drive', 'samsung', 'night',
       'audio', '720p', '850', 'capture', 'motion', 'surveillance',
       'white', 'bluetooth', '3g', 'auto', 'warranty', 'flash', 'new',
       'images', 'remote', 'hdmi', 'smart', 'usb', 'tv', 'kit', '12',
       'time', 'gsm', 'core', 'color', 'security', 'wireless', 'gb',
       'dual', 'recording', 'touch', 'wi', 'includes', 'angle', 'wide',
       'sd', 'cmos', 'fi', 'mobile', '10', 'quality', 'included', '1080p',
       'android', 'built', 'processor', 'unlocked', 'led', '16', 'battery',